In [22]:
import re
from collections import defaultdict, Counter
from math import prod, lcm

from functools import cache

import black
import jupyter_black
from parse import parse

jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)


def ints(text: str) -> list[int]:
    return [int(x) for x in re.findall("-?\d+", text)]


def first(iterable):
    return next(iter(iterable))


def data(day: int, parser=str, sep="\n", example=False) -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    filename = f"2023/{day}-example.txt" if example else f"2023/{day}.txt"
    sections = open(filename).read().rstrip().split(sep)
    return [parser(section) for section in sections]

In [94]:
# Day 1: Trebuchet?!
numbers = {
    "one": "1",
    "two": "2",
    "three": "3",
    "four": "4",
    "five": "5",
    "six": "6",
    "seven": "7",
    "eight": "8",
    "nine": "9",
}


def calibration_value(line, part1=True):
    numbers_found = ""
    for start, char in enumerate(line):
        if char.isdigit():
            numbers_found += char
        if part1:
            continue
        for key, value in numbers.items():
            if line.startswith(key, start):
                numbers_found += value
    calibration_value = numbers_found[0] + numbers_found[-1]
    return int(calibration_value)


games = data(1)
points = sum([calibration_value(line) for line in games])
print(f"Part 1: {points}")  # 53921
points = sum([calibration_value(line, part1=False) for line in games])
print(f"Part 2: {points}")  # 54676

Part 1: 53921
Part 2: 54676


In [151]:
# Day 2: Cube Conundrum
games = data(2)
possible_games = 0
powers = []
for game_nr, game in enumerate(games, 1):
    max_cubes_seen = {"red": 0, "blue": 0, "green": 0}
    cubes_seen = game.split(": ")[1].split("; ")
    for cubes in cubes_seen:
        for cube in cubes.split(", "):
            count, color = cube.split()
            count = int(count)
            if count > max_cubes_seen[color]:
                max_cubes_seen[color] = count
    if (
        max_cubes_seen["red"] <= 12
        and max_cubes_seen["green"] <= 13
        and max_cubes_seen["blue"] <= 14
    ):
        possible_games += game_nr
    powers.append(prod(max_cubes_seen.values()))

print(f"Part 1: {possible_games}")  # 2563
print(f"Part 2: {sum(powers)}")  # 70768

Part 1: 2563
Part 2: 70768


In [181]:
# Day 3: Gear Ratios
def neighbors(point):
    row, col = point
    potential = (
        (row - 1, col - 1),
        (row, col - 1),
        (row + 1, col - 1),
        (row - 1, col),
        (row + 1, col),
        (row - 1, col + 1),
        (row, col + 1),
        (row + 1, col + 1),
    )
    for (r, c) in potential:
        if 0 <= r < len(grid[0]) and 0 <= c < len(grid):
            yield (r, c)


grid = data(3)
gears = defaultdict(list)
part_numbers = []

for row_number, row in enumerate(grid):
    current_number = ""
    to_check = set()
    # Add '.' to catch numbers at end of line and avoid separate check after loop
    # completion
    for col_number, char in enumerate(row + "."):
        if char.isdigit():
            current_number += char
            for point in neighbors((row_number, col_number)):
                to_check.add(point)
        elif current_number:
            for (r, c) in to_check:
                if grid[r][c] not in "0123456789.":
                    # Symbol found, we are a part_number
                    part_numbers.append(int(current_number))
                if grid[r][c] == "*":
                    # Gear found, add number connected to this gear
                    gears[(r, c)].append(int(current_number))
            current_number = ""
            to_check = set()
print(f"Part 1: {sum(part_numbers)}")  # 514969
part2 = sum((prod(gear) for gear in gears.values() if len(gear) == 2))
print(f"Part 2: {part2}")  # 78915902

Part 1: 514969
Part 2: 78915902


In [94]:
# Day 4: Scratchcards
cards = data(4)
points = 0
total_cards = Counter([*range(len(cards))])  # One of each card to begin with
for card_nr, card in enumerate(cards):
    winning_numbers, my_numbers = map(ints, card.split(": ")[1].split(" | "))
    winners = sum(1 for number in my_numbers if number in winning_numbers)
    if winners:
        points += 2 ** (winners - 1)
        total_cards.update(
            total_cards[card_nr] * [*range(card_nr + 1, card_nr + winners + 1)]
        )

print(f"Part 1: {points}")  # 28750
print(f"Part 2: {sum(total_cards.values())}")  # 10212704

Part 1: 28750
Part 2: 10212704


In [95]:
# Day 5: If You Give A Seed A Fertilizer
def get_destination(seed, mapping):
    for dest, source, range_ in zip(mapping[0::3], mapping[1::3], mapping[2::3]):
        if source <= seed <= source + range_:
            return dest - source + seed
    return seed


def get_location(seed, mappings):
    for mapping in mappings:
        seed = get_destination(seed, mapping)
    return seed


chunks = data(5, sep="\n\n")
seeds = ints(chunks[0])
mappings = [tuple(ints(chunk)) for chunk in chunks[1:]]

print(f"Part 1: {min(get_location(seed, mappings) for seed in seeds)}")

# -- Part 2 -- Yucky.
def new_ranges(low, high, dest, source, range):
    """Return two lists (lows and highs) based on a continuous range from low to high and
    a mapping and then two lists including remaining low-high pairs to check (one list
    includes lows, one includes highs)"""
    offset = dest - source
    new_low = source
    new_high = source + range - 1
    # print(f"{new_low=} {new_high=}")
    if low >= new_low and high <= new_high:
        # Range intact, return low + offset, high + offset
        # print(f"Complete overlap {offset=}")
        return [low + offset], [high + offset], [], []
    if low <= source and new_low <= high <= new_high:
        # -------low---------------------high------------
        #         |                       |
        #             |------offset-------|
        #             |                         |
        # ---------new_low-----------------new_high------
        # print(f"Partial overlap, case 1 {offset=}")
        return (
            [low, new_low + offset],
            [new_low - 1, high + offset],
            [low],
            [new_low - 1],
        )
    if low < new_low and high > new_high:
        # -------low----------------------------high-------
        #         |                              |
        #              |------offset-------|
        #              |                   |
        # ----------new_low------------new_high------------
        # print(f"Partial overlap, case 2 {offset=}")
        return (
            [low, new_low + offset, new_high + 1],
            [
                new_low - 1,
                new_high + offset,
                high,
            ],
            [low, new_high + 1],
            [new_low - 1, high],
        )
    if new_low <= low <= new_high and high > new_high:
        # -------low---------------------high-----------
        #         |                       |
        #         |------offset-------|
        #      |                      |
        # ---new_low--------------new_high--------------
        # print(f"Partial overlap, case 3 {offset=}")
        return (
            [low + offset, new_high + 1],
            [
                new_high + offset,
                high,
            ],
            [new_high + 1],
            [high],
        )
    # Non overlapping
    # print(f"No overlap")
    return [], [], [], []


def apply_mapping(low, high, mapping):
    new_lows, new_highs = [], []
    # print(f"    {low=}     {high=}")
    for dest, source, range in zip(mapping[0::3], mapping[1::3], mapping[2::3]):
        lows_remaining = [low]
        highs_remaining = [high]
        while len(lows_remaining) and len(highs_remaining):
            l, h, lows_remaining, highs_remaining = new_ranges(
                lows_remaining[0], highs_remaining[0], dest, source, range
            )
            new_lows += l
            new_highs += h
            # print(lows_remaining, highs_remaining)
    if not new_lows:
        new_lows = [low]
    if not new_highs:
        new_highs = [high]
    # print(f"{new_lows=} {new_highs=}")
    return new_lows, new_highs


chunks = data(5, sep="\n\n")
ranges = ints(chunks[0])
mappings = [ints(chunk) for chunk in chunks[1:]]
input_lows = ranges[0::2]
input_highs = [low + length - 1 for low, length in zip(input_lows, ranges[1::2])]

locations = []
for lows, highs in zip(input_lows, input_highs):
    lows = [lows]
    highs = [highs]
    for mapping in mappings:
        # print("--------New mapping---------")
        # print(f"{mapping=}")
        new_lows, new_highs = [], []
        for low, high in zip(lows, highs):
            l, h = apply_mapping(low, high, mapping)
            new_lows += l
            new_highs += h
        # assert all(new_lows)
        lows = new_lows
        highs = new_highs
    locations.append(min(lows))

print(locations)

# 2520479 is the right answer, but by god this was an awful hack. The zeroes for the last
# ranges doesn't make sense to me, but I'm tired of this day now.

Part 1: 462648396
[4111139999, 942937748, 517161737, 3475769331, 2520479, 450060781, 56747007, 0, 0, 0]


In [217]:
# Day 6: Wait For It
times, distances = data(6, ints)

# Travel distance = speed * (time - button_press), where speed = button_press
part1 = prod(
    sum(
        1
        for button_press in range(time)
        if button_press * (time - button_press) > distance
    )
    for time, distance in zip(times, distances)
)
print(f"Part 1: {part1}")  # 512295

time = int("".join(map(str, times)))
distance = int("".join(map(str, distances)))
part2 = sum(
    1 for button_press in range(time) if button_press * (time - button_press) > distance
)
print(f"Part 2: {part2}")  # 36530883 (6s)

Part 1: 512295
Part 2: 36530883


In [278]:
# Day 7: Camel Cards
def most_common_character(s):
    s = sorted(s, reverse=True)
    freq = Counter(s)
    # Sort the characters by frequency
    sorted_chars = sorted(freq, key=freq.get, reverse=True)
    return sorted_chars[0]


lines = data(7)

hands = []
for line in lines:
    text, bid = line.split()
    # Mapping A, K, Q, J, T -> e, d, c, b, a
    hand = text.translate(str.maketrans("AKQJT", "edcba"))
    if hand == "bbbbb":
        pretend_hand = "22222"
    else:
        # replace all Jokers ('b') with the most common character *excluding* b
        pretend_hand = hand.replace("b", most_common_character(hand.replace("b", "")))
    hands.append((pretend_hand, hand, int(bid)))

# Part 1
decorated = [
    (
        sorted(Counter(hand).values(), reverse=True),
        hand,
        bid,
    )
    for _, hand, bid in hands
]
decorated.sort()
print(f"Part 1: {sum(rank * item[-1] for rank, item in enumerate(decorated, 1))}")

# Part 2
decorated = [
    (
        sorted(Counter(pretend_hand).values(), reverse=True),
        hand.replace("b", "1"),  # Jokers are worth the least
        bid,
    )
    for pretend_hand, hand, bid in hands
]
decorated.sort()
# 244848487
print(f"Part 2: {sum(rank * item[-1] for rank, item in enumerate(decorated, 1))}")

Part 1: 246409899
Part 2: 244848487


In [47]:
# Day 8: Haunted Wasteland
chunks = data(8, sep="\n\n")
instructions = [int(step.replace("R", "1").replace("L", "0")) for step in chunks[0]]
hops = {
    line.split(" = ")[0]: line.split(" = ")[1].strip("()").split(", ")
    for line in chunks[1].splitlines()
}

# Part 1
steps = 0
node = "AAA"
while node != "ZZZ":
    node = hops[node][instructions[steps % len(instructions)]]
    steps += 1
print(f"Part 1: {steps}")  # 19667

# Part 2
nodes = [start for start in hops if start[-1] == "A"]
# Find the frequencies for the different starting points ending with 'A'
freqs = []
steps = 0
while len(freqs) < len(nodes):
    for pos, node in enumerate(nodes):
        next = hops[node][instructions[steps % len(instructions)]]
        if next[-1] == "Z":
            freqs.append(steps + 1)
        nodes[pos] = next
    steps += 1
# Least Common Multiple is when all the frequencies match up
print(f"Part 2: {lcm(*freqs)}")  # 19185263738117

Part 1: 19667
Part 2: 19185263738117


In [74]:
# Day 9: Mirage Maintenance
def differences(numbers):
    return [b - a for a, b in zip(numbers, numbers[1:])]


def predict(value):
    prediction = [value]
    while any(prediction[-1]):
        prediction.append(differences(prediction[-1]))
    return sum(value[-1] for value in prediction)


values = data(9, ints)
print(f"Part 1: {sum(predict(value) for value in values)}") # 1834108701

Part 1: 1834108701
